# Jacobian Comparison: CasADi vs PyBullet

This notebook compares the Jacobian of a 2-DoF planar robot computed symbolically with CasADi vs. numerically with PyBullet.
The goal is to ensure both implementations are consistent at arbitrary joint configurations.

In [14]:
# Imports
import numpy as np
import casadi as ca
import pybullet as p
import pybullet_data
import pandas as pd
from urdfpy import URDF
from math import sin, cos
import matplotlib.pyplot as plt


# === CONFIG ===
NUM_TRAJECTORIES = 50
TRAJECTORY_LEN = 498  # 498 control actions, 499 states
STATE_DIM = 4
ACTION_DIM = 2

In [72]:
# Load URDF and define constants (these must match the values from your system)
URDF_FILE = "../../models/2dof_planar_robot.urdf"  # Adjust path if needed
# === LOAD FILES ===
state_file = "/home/jeauscq/Desktop/jcq_thesis/datasets/Policy/MPC/unconst/mpc_generated_dataset.csv"
action_file = "/home/jeauscq/Desktop/jcq_thesis/datasets/Policy/MPC/unconst/mpc_generated_actions.csv"

robot = URDF.load(URDF_FILE)

link1 = next(link for link in robot.links if link.name == "link1")
link2 = next(link for link in robot.links if link.name == "link2")

L1 = link1.visuals[0].geometry.box.size[0]
L2 = link2.visuals[0].geometry.box.size[0]

In [73]:
def load_data(path, num_trajectories, expected_len):
    data = []
    with open(path, "r") as f:
        for i, line in enumerate(f):
            if i >= num_trajectories:
                break
            parts = line.strip().split(",")
            traj = [list(map(float, entry.split(";"))) for entry in parts]
            assert len(traj) == expected_len, f"Trajectory length mismatch in line {i}, obtained {len(traj)}, expected {expected_len}"
            data.append(np.array(traj))
    return np.array(data)

In [74]:
states = load_data(state_file, NUM_TRAJECTORIES, TRAJECTORY_LEN + 1)
actions = load_data(action_file, NUM_TRAJECTORIES, TRAJECTORY_LEN)

In [55]:
# Helper to compute CasADi-based force
def build_jacobian_casadi_function():
    theta1 = ca.MX.sym("theta1")
    theta2 = ca.MX.sym("theta2")

    l1 = 1.0
    l2 = 1.0
    s1 = ca.sin(theta1)
    c1 = ca.cos(theta1)
    s12 = ca.sin(theta1 + theta2)
    c12 = ca.cos(theta1 + theta2)

    J = ca.MX.zeros(2, 2)
    J[0, 0] = -l1 * s1 - l2 * s12
    J[0, 1] = -l2 * s12
    J[1, 0] = l1 * c1 + l2 * c12
    J[1, 1] = l2 * c12

    J[1, :] *= -1
    # Return both force and full Jacobian
    return ca.Function("Jac", [theta1, theta2], [J])

compute_jacobian_casadi = build_jacobian_casadi_function()


In [56]:
# Regular solve (no regularization)
def build_force_function_unregularized():
    J = ca.MX.sym("J", 2, 2)
    tau = ca.MX.sym("tau", 2, 1)
    F = ca.solve(J.T, tau)
    return ca.Function("F_e_unreg", [J, tau], [F])


In [34]:
# Regularized solve
def build_force_function_regularized(lambda_reg=1e-6):
    J = ca.MX.sym("J", 2, 2)
    tau = ca.MX.sym("tau", 2, 1)
    JTJ = J.T @ J
    F = ca.solve(JTJ + lambda_reg * ca.MX.eye(JTJ.size1()), J.T @ tau)
    return ca.Function("F_e_reg", [J, tau], [F])

In [35]:
# Instantiate both
casadi_force_fn_unreg = build_force_function_unregularized()
casadi_force_fn_reg = build_force_function_regularized()

In [36]:
# PyBullet setup
client = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

robot_id = p.loadURDF(URDF_FILE, basePosition=(0, 0, 0.07),
                      baseOrientation=p.getQuaternionFromEuler((0, 3.14, 0)),
                      useFixedBase=True)

# Identify joint indices
joint_indices = [i for i in range(p.getNumJoints(robot_id)) if p.getJointInfo(robot_id, i)[2] == p.JOINT_REVOLUTE]

In [37]:
# Helper to compute PyBullet-based end-effector force
def compute_force_pybullet(theta1, theta2, dtheta1, dtheta2, torque):
    p.resetJointState(robot_id, 0, theta1, dtheta1)
    p.resetJointState(robot_id, 1, theta2, dtheta2)
    zero_acc = [0, 0]
    jac_t, _ = p.calculateJacobian(robot_id, linkIndex=2, localPosition=[1, 0, 0], objPositions=[theta1, theta2], objVelocities=[dtheta1, dtheta2], objAccelerations=zero_acc)
    J = np.array(jac_t)[[0, 2], :]  # Only translational part

    J_pinv = np.linalg.pinv(J)

    return J_pinv.T @ torque, J  # Force from torque = (J^T)^+ * tau

In [ ]:
def build_force_function_regularized_adaptive():
    J = ca.MX.sym("J", 2, 2)
    tau = ca.MX.sym("tau", 2, 1)
    lambda_reg = ca.MX.sym("lambda_reg")  # Now a runtime input
    JTJ = J.T @ J
    F = ca.solve(JTJ + lambda_reg * ca.MX.eye(JTJ.size1()), J.T @ tau)
    return ca.Function("F_e_reg_adaptive", [J, tau, lambda_reg], [F])

casadi_force_fn_reg_adaptive = build_force_function_regularized_adaptive()

In [ ]:
# Compare forces and Jacobians
pyb_forces = []

for traj_idx in range(2):
    for step in range(498):
        state = states[traj_idx, step]
        torque = actions[traj_idx, step]
        theta1, theta2 = state[0], state[1]
        dtheta1, dtheta2 = state[2], state[3]

        F_pybullet, J_pybullet = compute_force_pybullet(theta1, theta2, dtheta1, dtheta2, torque)
        F_pybullet = np.linalg.norm(F_pybullet)

        # Get symbolic Jacobian from CasADi
        J_casadi = compute_jacobian_casadi(theta1, theta2)  # assumed to return np.array
        cond = np.linalg.cond(J_casadi)
        lambda_reg = min(1.0, 1e-4 * cond**2)
        F_casadi = casadi_force_fn_reg_adaptive(J_casadi_mx, tau_mx, ca.DM(lambda_reg))


        # Choose function based on condition number
        J_casadi_mx = ca.DM(J_casadi)
        tau_mx = ca.DM(torque).reshape((2, 1))
        if cond < 0.01:
            print(f"Condition number too low: {cond:.5f} in traj {traj_idx} and step {step}")
            F_casadi = casadi_force_fn_reg(J_casadi_mx, tau_mx)
        if cond > 100:
            print(f"Condition number too high: {cond:.5f} in traj {traj_idx} and step {step}")
            F_casadi = casadi_force_fn_reg(J_casadi_mx, tau_mx)
        else:
            F_casadi = casadi_force_fn_unreg(J_casadi_mx, tau_mx)

        F_casadi = F_casadi.full().flatten()
        F_casadi = np.linalg.norm(F_casadi)

        pyb_forces.append(F_pybullet)

        if F_casadi > 100:
            print(f"Force too high: {F_casadi:.5f} in traj {traj_idx} and step {step}")
            print(f"PyBullet force: {F_pybullet:.5f}, CasADi force: {F_casadi:.5f}")


        if abs(F_pybullet - F_casadi) > 0.01:
            print(f"Concerning difference of {F_pybullet-F_casadi:.5f} in traj {traj_idx} and step {step}")
            print(f"PyBullet force: {F_pybullet:.5f}, CasADi force: {F_casadi:.5f}")
    print(f"Finished trajectory {traj_idx}.")

Force too high: 121.56720 in traj 0 and step 177
PyBullet force: 121.56720, CasADi force: 121.56720
Force too high: 124.25063 in traj 0 and step 197
PyBullet force: 124.25063, CasADi force: 124.25063
Force too high: 100.93910 in traj 0 and step 427
PyBullet force: 100.93910, CasADi force: 100.93910
Force too high: 105.47537 in traj 0 and step 428
PyBullet force: 105.47537, CasADi force: 105.47537
Force too high: 110.57035 in traj 0 and step 429
PyBullet force: 110.57035, CasADi force: 110.57035
Force too high: 116.32484 in traj 0 and step 430
PyBullet force: 116.32484, CasADi force: 116.32484
Force too high: 122.86252 in traj 0 and step 431
PyBullet force: 122.86252, CasADi force: 122.86252
Force too high: 130.34575 in traj 0 and step 432
PyBullet force: 130.34575, CasADi force: 130.34575
Force too high: 138.98145 in traj 0 and step 433
PyBullet force: 138.98145, CasADi force: 138.98145
Force too high: 149.04592 in traj 0 and step 434
PyBullet force: 149.04592, CasADi force: 149.04592
